# Fine-tuning a pretrained model
<https://huggingface.co/docs/transformers/v4.14.1/en/training>


In [1]:
from datasets import load_dataset
raw_datasets = load_dataset("imdb")

/Users/kinwong/mambaforge/envs/pt-hf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset imdb (/Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 110.62it/s]


The raw_datasets object is a dictionary with three keys: "train", "test" and "unsupervised" (which correspond to the three splits of that dataset). We will use the "train" split for training and the "test" split for validation.

To preprocess our data, we will need a tokenizer:

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-7ed6e90bbf90227c.arrow
Loading cached processed dataset at /Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-cf3c4992ffb060a8.arrow
Loading cached processed dataset at /Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-3ea9f1c7bff5a71d.arrow


You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets documentation.

Next we will generate a small subset of the training and validation set, to enable faster training:

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"],

Loading cached shuffled indices for dataset at /Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-f00c281fb7fc6979.arrow
Loading cached shuffled indices for dataset at /Users/kinwong/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-c71fd02c141f1267.arrow


In all the examples below, we will always use small_train_dataset and small_eval_dataset. Just replace them by their full equivalent to train or evaluate on the full dataset.

Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a Trainer API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like logging, gradient accumulation, and mixed precision.

First, let’s define our model:

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

This will issue a warning about some of the pretrained weights not being used and some weights being randomly initialized. That’s because we are throwing away the pretraining head of the BERT model to replace it with a classification head which is randomly initialized. We will fine-tune this model on our task, transferring the knowledge of the pretrained model to it (which is why doing this is called transfer learning).

Then, to define our Trainer, we will need to instantiate a TrainingArguments. This class contains all the hyperparameters we can tune for the Trainer or the flags to activate the different training options it supports. Let’s begin by using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", use_mps_device=True)

Then we can instantiate a Trainer like this:

In [7]:

from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)
print(f'place_model_on_device = {trainer.place_model_on_device}, is_model_parallel = {trainer.is_model_parallel}')


place_model_on_device = True, is_model_parallel = False


To fine-tune our model, we just need to call

In [8]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/kinwong/mambaforge/envs/pt-hf/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 108311810
  3%|▎         | 13/375 [02:30<1:24:17, 13.97s/it]

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete (as long as you have access to a GPU). It won’t actually tell you anything useful about how well (or badly) your model is performing however as by default, there is no evaluation during training, and we didn’t tell the Trainer to compute any metrics. Let’s have a look on how to do that now!

To have the Trainer compute and report metrics, we need to give it a compute_metrics function that takes predictions and labels (grouped in a namedtuple called EvalPrediction) and return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the load_metric function. here we simply use accuracy. Then we define the compute_metrics function that just convert logits to predictions (remember that all 🤗 Transformers models return the logits) and feed them to compute method of this metric.

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)